## Fourier Neural Operator 1D

In [ ]:
## Imports 
import matplotlib.pyplot as plt 
import numpy as np 
import torch 
import torch.nn as nn
from timeit import default_timer

In [ ]:
## Set seeds
torch.manual_seed(0)
np.random.seed(0)

## Create Data

### Heat Equation:

\begin{align*}
\frac{\partial u}{\partial t} &= \alpha \frac{\partial^2 u}{\partial x^2} \\
u(x, 0) &= x^2 \\
u(0, t) &= u(1, t) = 2 - \exp(-t)
\end{align*}

Solution: 


In [ ]:
## TODO: Create Heat Equation Class

In [ ]:
## Steps in x direction
nx = 100
## Steps in t direction
nt = 1000

h = 1 / nx ## Step size in x direction
k = 1 / nt ## Step size in t direction

alpha = 0.1
r =  alpha * k / h**2

X, T = np.meshgrid(np.linspace(0, 1, nx + 1), np.linspace(0, 1, nt + 1))

plt.scatter(X, T)
plt.show()

In [ ]:
## Initial Condition
beta = np.pi
u0 = np.sin(beta * X[0]) 

## Boundary Conditions
ux0 = u0[0] # T[:, 0]
uxn = u0[-1] # 2 - np.exp(-T[:, -1])

In [ ]:
## Create Solution Matrix
u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

In [ ]:
A = np.zeros((nx - 1, nx - 1))
B = np.zeros((nx - 1, nx - 1))

for i in range(nx - 1):
    A[i, i] = 2 + 2 * r
    B[i, i] = 2 - 2 * r

for i in range(nx - 2):
    A[i + 1, i] = -r
    A[i, i + 1] = -r
    B[i + 1, i] = r
    B[i, i + 1] = r

In [ ]:
## Compute Solution
Ainv = np.linalg.inv(A)

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

In [ ]:
plt.imshow(u, aspect='auto')
plt.colorbar()
plt.show()

In [ ]:
## Number of Samples 
n = 10000
## Output
data_in = []
data_out = []

## Generate Data
for _ in range(n):
    beta = np.random.uniform(0, 10)
    beta_vec = beta * np.ones(101)

    u0 = np.sin(beta * X[0])

    data_in.append([beta_vec, u0])

    ## Create Solution Matrix
    u = np.zeros((nt + 1, nx + 1))
    u[0] = u0
    u[:, 0] = ux0
    u[:, -1] = uxn

    b = np.zeros(nx - 1)
    for j in range(1, nt + 1):
        b[0] = r * u[j - 1, 0] + r * u[j, 0]
        b[-1] = r * u[j - 1, -1] + r * u[j, -1]
        u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

    data_out.append(u[-1])

In [ ]:
data_in = np.array(torch.tensor(data_in).float())
data_out = np.array(torch.tensor(data_out).float().unsqueeze(1))

In [ ]:
## Data Loader
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, t, x):
        self.t = t
        self.x = x

    def __len__(self):
        return len(self.t)
    
    def __getitem__(self, index):
        return self.t[index], self.x[index]

data = CustomDataset(data_in, data_out)

In [ ]:
data[0][0]

In [ ]:
data[0][0].shape

In [ ]:
dataloader = DataLoader(data, batch_size=64, shuffle=True)

## Build Network

In [ ]:
## Get Device for Training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device.')

In [ ]:
## Fourier Integral Kernel 1D
class FourierIntegralKernel1D(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, modes: int):
        super(FourierIntegralKernel1D, self).__init__()
        '''
        '''
        self.in_channels = in_channels
        self.out_channels = out_channels 
        self.modes = modes 
        ## Set (random) weights for the linear transform
        weights = torch.rand(self.modes, self.out_channels, self.in_channels, dtype=torch.cfloat) 
        self.weights = nn.Parameter(weights / (self.in_channels * self.out_channels)) ## Optional: Scale weights

    def forward(self, v: torch.Tensor) -> torch.Tensor:
        '''
        FFT -> Linear Transform -> Inverse FFT
        '''
        ## FFT
        v_rfft = torch.fft.rfft(v) 

        ## Linear Transform 
        lv_rfft = torch.zeros(v_rfft.shape, dtype=torch.cfloat)
        lv_rfft[:, :, :self.modes] = torch.einsum('koi, bki -> bko', self.weights, v_rfft[:, :, :self.modes].permute(0, 2, 1)).permute(0, 2, 1) ## TODO: Should I have 5 dimensions here?
        
        ## Inverse FFT
        v2 = torch.fft.irfft(lv_rfft, n=v.shape[-1])
        return v2
        

In [ ]:
## Fourier Network Operator 1D
class FourierNetworkOperator1D(nn.Module):
    def __init__(self, da: int, du: int, width: int, modes: int):
        super(FourierNetworkOperator1D, self).__init__()
        '''
        '''
        self.width = width
        self.modes = modes

        ## P: Lifts the lower dimensional function to higher dimensional space
        self.P = nn.Conv1d(da, self.width, 1) ## TODO: Change da

        ## K: Fourier integral kernel operator
        self.k0 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        self.k1 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        self.k2 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        self.k3 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        # self.k4 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        # self.k5 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        # self.k6 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        # self.k7 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        # self.k8 = FourierIntegralKernel1D(self.width, self.width, self.modes)
        # self.k9 = FourierIntegralKernel1D(self.width, self.width, self.modes)

        ## W: Pointwise linear operator
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)
        # self.w4 = nn.Conv1d(self.width, self.width, 1)
        # self.w5 = nn.Conv1d(self.width, self.width, 1)
        # self.w6 = nn.Conv1d(self.width, self.width, 1)
        # self.w7 = nn.Conv1d(self.width, self.width, 1)
        # self.w8 = nn.Conv1d(self.width, self.width, 1)
        # self.w9 = nn.Conv1d(self.width, self.width, 1)

        ## Q: Projects the higher dimensional function to lower dimensional space
        self.Q = nn.Conv1d(self.width, du, 1) ## TODO: Change du

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        '''
        ## P
        x = self.P(x)

        ## Fourier Layer #0
        ## K
        x1 = self.k0(x)
        ## W
        x2 = self.w0(x)
        ## Sum
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)
        # x = nn.functional.gelu(x1)
        # x = nn.functional.gelu(x2)

        ## Fourier Layer #1
        ## K
        x1 = self.k1(x)
        ## W
        x2 = self.w1(x)
        ## Sum 
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)
        # x = nn.functional.gelu(x1)
        # x = nn.functional.gelu(x2)

        ## Fourier Layer #2
        ## K
        x1 = self.k2(x)
        ## W
        x2 = self.w2(x)
        ## Sum
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)
        # x = nn.functional.gelu(x1)
        # x = nn.functional.gelu(x2)

        ## Fourier Layer #3
        ## K
        x1 = self.k3(x)
        ## W
        x2 = self.w3(x)
        ## Sum
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)
        # x = nn.functional.gelu(x1)
        # x = nn.functional.gelu(x2)

        # ## Fourier Layer #4
        # ## K
        # x1 = self.k4(x)
        # ## W
        # x2 = self.w4(x)
        # ## Sum
        # x = x1 + x2
        # ## Gelu
        # x = nn.functional.gelu(x)
        # # x = nn.functional.gelu(x1)
        # # x = nn.functional.gelu(x2)

        # ## Fourier Layer #5
        # ## K
        # x1 = self.k5(x)
        # ## W
        # x2 = self.w5(x)
        # ## Sum
        # x = x1 + x2
        # ## Gelu
        # x = nn.functional.gelu(x)
        # # x = nn.functional.gelu(x1)
        # # x = nn.functional.gelu(x2)

        # ## Fourier Layer #6
        # ## K
        # x1 = self.k6(x)
        # ## W
        # x2 = self.w6(x)
        # ## Sum
        # x = x1 + x2
        # ## Gelu
        # x = nn.functional.gelu(x)
        # # x = nn.functional.gelu(x1)
        # # x = nn.functional.gelu(x2)

        # ## Fourier Layer #7
        # ## K
        # x1 = self.k7(x)
        # ## W
        # x2 = self.w7(x)
        # ## Sum
        # x = x1 + x2
        # ## Gelu
        # x = nn.functional.gelu(x)
        # # x = nn.functional.gelu(x1)
        # # x = nn.functional.gelu(x2)

        # ## Fourier Layer #8
        # ## K
        # x1 = self.k8(x)
        # ## W
        # x2 = self.w8(x)
        # ## Sum
        # x = x1 + x2
        # ## Gelu
        # x = nn.functional.gelu(x)
        # # x = nn.functional.gelu(x1)
        # # x = nn.functional.gelu(x2)

        # ## Fourier Layer #9
        # ## K
        # x1 = self.k9(x)
        # ## W
        # x2 = self.w9(x)
        # ## Sum
        # x = x1 + x2
        # ## Gelu
        # x = nn.functional.gelu(x)
        # # x = nn.functional.gelu(x1)
        # # x = nn.functional.gelu(x2)

        ## Q
        x = self.Q(x)
        return x


In [ ]:
import operator
from functools import reduce

def count_parameters(model):
    c = 0
    for p in list(model.parameters()):
        c += reduce(operator.mul, list(p.size() + (2, ) if p.is_complex() else p.size()))
    return c

model = FourierNetworkOperator1D(2, 1, width=64, modes=4)
print(f'Number of parameters: {count_parameters(model)}')

## Train

In [ ]:
## Model Parameters
learning_rate = 1e-3
epochs = 5

In [ ]:
## Loss Function
loss_function = nn.MSELoss()

In [ ]:
## Optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
## Test 0
beta0 = np.random.uniform(0, 10)
print(f'beta: {beta0}')
beta0_vec = torch.from_numpy(beta0 * np.ones(101)).float().unsqueeze(0).unsqueeze(0)

u0 = torch.from_numpy(np.sin(beta0 * X[0])).float().unsqueeze(0).unsqueeze(0)

u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

plt.scatter(np.linspace(0, 1, nx + 1), u[-1], alpha=0.5)
plt.scatter(np.linspace(0, 1, nx + 1), model(torch.cat((beta0_vec, u0), 1)).detach())
plt.show()

In [ ]:
## Test 1
beta1 = np.random.uniform(0, 10)
print(f'beta: {beta1}')
beta1_vec = torch.from_numpy(beta1 * np.ones(101)).float().unsqueeze(0).unsqueeze(0)

u0 = torch.from_numpy(np.sin(beta1 * X[0])).float().unsqueeze(0).unsqueeze(0)

u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

plt.scatter(np.linspace(0, 1, nx + 1), u[-1], alpha=0.5)
plt.scatter(np.linspace(0, 1, nx + 1), model(torch.cat((beta1_vec, u0), 1)).detach())
plt.show()

In [ ]:
## Test 2
beta2 = np.random.uniform(0, 10)
print(f'beta: {beta2}')
beta2_vec = torch.from_numpy(beta2 * np.ones(101)).float().unsqueeze(0).unsqueeze(0)

u0 = torch.from_numpy(np.sin(beta2 * X[0])).float().unsqueeze(0).unsqueeze(0)

u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

plt.scatter(np.linspace(0, 1, nx + 1), u[-1], alpha=0.5)
plt.scatter(np.linspace(0, 1, nx + 1), model(torch.cat((beta2_vec, u0), 1)).detach())
plt.show()

In [ ]:
## Training Loop
def train_loop(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    for batch, (t, x) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(t)
        loss = loss_function(pred, x)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(t)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloader, model, loss_function, optimizer)
print("Done!")

## Test

In [ ]:
## Test 0
u0 = torch.from_numpy(np.sin(beta0 * X[0])).float().unsqueeze(0).unsqueeze(0)

u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

plt.scatter(np.linspace(0, 1, nx + 1), u[-1], alpha=0.5)
plt.scatter(np.linspace(0, 1, nx + 1), model(torch.cat((beta1_vec, u0), 1)).detach())
plt.show()

In [ ]:
## Test 1
u0 = torch.from_numpy(np.sin(beta1 * X[0])).float().unsqueeze(0).unsqueeze(0)

u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

plt.scatter(np.linspace(0, 1, nx + 1), u[-1], alpha=0.5)
plt.scatter(np.linspace(0, 1, nx + 1), model(torch.cat((beta1_vec, u0), 1)).detach())
plt.show()

In [ ]:
## Test 2
u0 = torch.from_numpy(np.sin(beta2 * X[0])).float().unsqueeze(0).unsqueeze(0)

u = np.zeros((nt + 1, nx + 1))
u[0] = u0
u[:, 0] = ux0
u[:, -1] = uxn

b = np.zeros(nx - 1)
for j in range(1, nt + 1):
    b[0] = r * u[j - 1, 0] + r * u[j, 0]
    b[-1] = r * u[j - 1, -1] + r * u[j, -1]
    u[j, 1:nx] = Ainv @ ((B @ u[j - 1, 1:nx]) + b)

plt.scatter(np.linspace(0, 1, nx + 1), u[-1], alpha=0.5)
plt.scatter(np.linspace(0, 1, nx + 1), model(torch.cat((beta2_vec, u0), 1)).detach())
plt.show()